<a href="https://colab.research.google.com/github/caleb-code/tictactoe/blob/main/Tic_Tac_Toe_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-agents
!pip uninstall tensorflow -y
!pip install tensorflow==2.15.0

In [ ]:
!export TF_USE_LEGACY_KERAS=1

In [ ]:
import tf_agents as tfa
from tf_agents.environments import tf_environment
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents import dqn
from tf_agents.utils import common
from tf_agents.policies import epsilon_greedy_policy
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

In [ ]:
import numpy as np
import tensorflow as tf
import keras

In [ ]:
def winner(state):
        if np.sum(np.abs(state)) >= 5:
          state = state.reshape(3,3)
          for i in range(0,3):
            if np.abs(np.sum(state[:, i])) == 3:
              return np.sum(state[:, i]) / 3
          for i in state:
            if np.abs(np.sum(i)) == 3:
              return np.sum(i) / 3
          if np.abs(np.sum(np.fliplr(state).diagonal())) == 3:
            return np.sum(np.fliplr(state).diagonal()) / 3
          if np.abs(np.sum(state.diagonal())) == 3:
            return np.sum(state.diagonal()) / 3
        if np.sum(np.abs(state)) == 9:
          return 0
        return None

def update_board(state, player, pos):
      if state[pos] == 0:
        state[pos] = player
      return state

In [ ]:
class TicTacToe(tfa.environments.py_environment.PyEnvironment):
    def __init__(self):
        self._observation_spec = array_spec.BoundedArraySpec(shape=(9,), dtype=np.int32, minimum=-1, maximum=1)
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=8)
        self._state = np.zeros(shape=(9,))
        self.episode_running = True
        self.player = 1

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec

    def _reset(self):
        self.player = 1
        self._state = np.zeros(shape=(9,))
        return ts.restart(self._state.astype(np.int32))

    def _step(self, action):
      if winner(self._state):
        reward = winner(self._state)
        return ts.termination(self._state.astype(np.int32), reward=abs(reward))
      self._state = update_board(self._state, self.player, action)
      self.player *= -1
      reward = winner(self._state)
      if reward == None:
        return ts.transition(self._state.astype(np.int32), reward=0, discount=1)
      else:
        return ts.termination(self._state.astype(np.int32), reward=reward*self.player)


NameError: name 'x' is not defined

In [ ]:
env  = tfa.environments.tf_py_environment.TFPyEnvironment(TicTacToe())

In [ ]:
model = QNetwork(
        input_tensor_spec=env.observation_spec(),
        action_spec=env.action_spec(),
        fc_layer_params=[9,9], # @param
        dropout_layer_params=[0.2,0.2], # @param
        activation_fn="relu" # @param
)
lr = 1 # @param {type:"number"}
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

agent = dqn.dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    optimizer = optimizer,
    q_network = model
)
print(model.summary())

Model: "QNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncodingNetwork (EncodingN  multiple                  180       
 etwork)                                                         
                                                                 
 dense_2 (Dense)             multiple                  90        
                                                                 
Total params: 270 (1.05 KB)
Trainable params: 270 (1.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
modelx = QNetwork(
        input_tensor_spec=env.observation_spec(),
        action_spec=env.action_spec(),
        fc_layer_params=(9,9), # @param
        dropout_layer_params=[0.2,0.2], # @param
        activation_fn="relu" # @param
)
lr = 1 # @param {type:"number"}
optimizer_x = tf.keras.optimizers.Adam(learning_rate=lr)

agentx = dqn.dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    optimizer = optimizer_x,
    q_network = modelx
)
print(modelx.summary())

Model: "QNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncodingNetwork (EncodingN  multiple                  180       
 etwork)                                                         
                                                                 
 dense_8 (Dense)             multiple                  90        
                                                                 
Total params: 270 (1.05 KB)
Trainable params: 270 (1.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
def filter_legal(q_values, observation):
  q_values = np.asarray(q_values).flatten()
  observation = np.asarray(observation).flatten()
  for i in range(len(q_values)):
    if observation[i] != 0:
      q_values[i] = -np.inf
  return np.argmax(q_values)

In [ ]:
print(env.reset())
print(env.step(5).is_last())

TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>})
tf.Tensor([False], shape=(1,), dtype=bool)


In [ ]:
def change_reward(time_step, reward):
  return ts.TimeStep(
    step_type=time_step.step_type,
    reward=reward,
    discount=time_step.discount,
    observation=time_step.observation
)

In [ ]:
epilson = 0.1
for j in range(0, 10000+1):
  time_step = env.reset()
  data1 = []
  data2 = []
  avg1 = []
  avg2 = []
  avg1_con = []
  avg2_con = []
  total1 = 0
  total2 = 0
  while time_step.is_last() != True:
    # PLAYER 1
    if np.random.rand() < epilson:
      q_values = np.random.rand(9)
    else:
      q_values = model.call(time_step.observation)[0]
      avg1_con.append(np.max(q_values)- np.min(q_values))
    move = [filter_legal(q_values, time_step.observation)]
    tf_action = tfa.trajectories.PolicyStep(tf.convert_to_tensor(move, dtype=tf.int32))
    next_time_step = env.step(tf_action.action)
    # CALCULATE PLAYER 2's PREVIOUS ROUND TRAINING DATA
    if time_step.is_first() == False:
      total2 += new_time_step.reward - next_time_step.reward
      x_training_data = tfa.trajectories.trajectory.Transition(new_time_step, tf_action_x, change_reward(time_step, new_time_step.reward - next_time_step.reward))
    # PLAYER 2 MOVES
    if np.random.rand() < epilson:
      x_values = np.random.rand(9)
    else:
      x_values = modelx.call(next_time_step.observation)[0]
      avg2_con.append(np.max(x_values)-np.min(x_values))
    move_x = [filter_legal(x_values, next_time_step.observation)]
    tf_action_x = tfa.trajectories.PolicyStep(tf.convert_to_tensor(move_x, dtype=tf.int32))
    new_time_step = env.step(tf_action_x.action)
    # CALCULATE PLAYER 1's TRAINING DATA
    total1 += next_time_step.reward - new_time_step.reward
    o_training_data = tfa.trajectories.trajectory.Transition(time_step, tf_action, change_reward(next_time_step, next_time_step.reward - new_time_step.reward))
    data1.append(o_training_data)
    if time_step.is_first() == False:
      data2.append(x_training_data)
    time_step = new_time_step
  for i in data1:
    avg1.append(agent.train(i).loss)
  for i in data2:
    avg2.append(agentx.train(i).loss)
  print("Loss O: " + str(sum(avg1)/len(avg1)))
  print("Loss X: " + str(sum(avg2)/len(avg2)))
  print("Reward O: " + str(total1))
  print("Reward X: " + str(total2))
  if len(avg1_con) and len(avg2_con):
    print("Confidence O: " + str(sum(avg1_con)+10e-10/len(avg1_con)))
    print("Confidence X: " + str(sum(avg2_con)+10e-10/len(avg2_con)))
  time_step = new_time_step
  print(time_step.observation.numpy().reshape(3,3))
  print("-------------------------------------" + str(j))

Loss O: tf.Tensor(77488620.0, shape=(), dtype=float32)
Loss X: tf.Tensor(9.308728, shape=(), dtype=float32)
Reward O: tf.Tensor([-2.], shape=(1,), dtype=float32)
Reward X: tf.Tensor([1.], shape=(1,), dtype=float32)
Confidence O: 652405472.0
Confidence X: 156.53790664706185
[[ 1  1  1]
 [-1  1 -1]
 [ 1 -1 -1]]
-------------------------------------0
Loss O: tf.Tensor(84244940.0, shape=(), dtype=float32)
Loss X: tf.Tensor(1.6008209, shape=(), dtype=float32)
Reward O: tf.Tensor([-2.], shape=(1,), dtype=float32)
Reward X: tf.Tensor([1.], shape=(1,), dtype=float32)
Confidence O: 813299700.0
Confidence X: 270.8692741396043
[[-1  1 -1]
 [ 1  1  1]
 [ 1 -1 -1]]
-------------------------------------1
Loss O: tf.Tensor(102380030.0, shape=(), dtype=float32)
Loss X: tf.Tensor(1.715172, shape=(), dtype=float32)
Reward O: tf.Tensor([-2.], shape=(1,), dtype=float32)
Reward X: tf.Tensor([1.], shape=(1,), dtype=float32)
Confidence O: 661623484.0
Confidence X: 281.4419555666062
[[-1  1 -1]
 [ 1  1  1]
 [

In [ ]:
time_step = env.reset()

In [ ]:
move = 4 #@param
time_step = env.step(move)
print(time_step)
print(move)
print(time_step.observation.numpy().reshape(3,3))

TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[ 0, -1,  0,  0,  1,  1,  0,  0,  0]], dtype=int32)>})
4
[[ 0 -1  0]
 [ 0  1  1]
 [ 0  0  0]]


In [ ]:
print(time_step)
q_values = model.call(time_step.observation)[0]
print(q_values)
move = np.expand_dims(filter_legal(q_values, time_step.observation),axis=0)
print(agent.collect_policy.action(time_step))
time_step = env.step(move)
print(tf.nn.softmax(q_values))
print(time_step.observation.numpy().reshape(3,3))

TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[ 0, -1,  0,  0,  1,  1,  0,  0,  0]], dtype=int32)>})
tf.Tensor(
[[ 2824486.   26064062.     320026.16   128624.93 18207994.   15938045.
  19260984.    1682464.6   1308460.5 ]], shape=(1, 9), dtype=float32)
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), info=())
tf.Tensor([[0. 1. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 9), dtype=float32)
[[ 0 -1  0]
 [ 0  1  1]
 [-1  0  0]]


In [ ]:
print(time_step)
q_values = modelx.call(time_step.observation)[0]
print(q_values)
move = np.expand_dims(filter_legal(q_values, time_step.observation),axis=0)
print(move)
time_step = env.step(move)
print(q_values)
print(time_step.observation.numpy().reshape(3,3))

TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[ 0, -1,  0,  0,  1,  1, -1,  0,  0]], dtype=int32)>})
tf.Tensor(
[[56.986214   5.4379573 56.860687  44.851173  23.666601  33.121403
  10.602286  57.414707  57.61726  ]], shape=(1, 9), dtype=float32)
[8]
tf.Tensor(
[[56.986214   5.4379573 56.860687  44.851173  23.666601  33.121403
  10.602286  57.414707  57.61726  ]], shape=(1, 9), dtype=float32)
[[ 0 -1  0]
 [ 0  1  1]
 [-1  0  1]]


[4]
